# 📊 09 – Building Interactive Dashboards with Plotly Dash

**Plotly Dash** is a powerful framework for creating interactive web applications directly in Python.  
It combines the expressiveness of **Plotly** visualizations with the simplicity of **Flask**,  
allowing you to build dashboards without needing any HTML, CSS, or JavaScript.

In this notebook, you will learn how to:
- Build a fully interactive dashboard using **Dash**
- Connect **callbacks** to link user inputs and charts dynamically
- Display **KPI metrics**, **interactive filters**, and **responsive graphs**
- Run and visualize your dashboard directly from this notebook

## 🧰 1️⃣ Import Required Libraries

We’ll start by importing all the necessary libraries, including:
- **Dash** and **Plotly Express** for building the dashboard
- **Pandas** for data manipulation
- **JupyterDash** to run Dash apps directly within Jupyter Notebook

In [25]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output

# Ensure data directory exists
os.makedirs("images", exist_ok=True)

# Load sample dataset
df = px.data.gapminder()

print("✅ Libraries loaded successfully and dataset ready!")
df.head()

✅ Libraries loaded successfully and dataset ready!


,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
0,Afghanistan,Asia,1952,28.801,8425333,779.445314,AFG,4
1,Afghanistan,Asia,1957,30.332,9240934,820.853030,AFG,4
2,Afghanistan,Asia,1962,31.997,10267083,853.100710,AFG,4
3,Afghanistan,Asia,1967,34.020,11537966,836.197138,AFG,4
4,Afghanistan,Asia,1972,36.088,13079460,739.981106,AFG,4


## 🌍 2️⃣ Exploring the Dataset

The Gapminder dataset contains country-level indicators across multiple years,  
including **life expectancy**, **GDP per capita**, and **population**.

Let's take a quick look at the dataset summary.

In [26]:
# Display dataset info
df.info()

# Show sample data
df.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1704 entries, 0 to 1703
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   country    1704 non-null   object 
 1   continent  1704 non-null   object 
 2   year       1704 non-null   int64  
 3   lifeExp    1704 non-null   float64
 4   pop        1704 non-null   int64  
 5   gdpPercap  1704 non-null   float64
 6   iso_alpha  1704 non-null   object 
 7   iso_num    1704 non-null   int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 106.6+ KB


,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
1225,Poland,Europe,1957,65.770,28235346,4734.253019,POL,616
561,Gambia,Africa,1997,55.861,1235767,653.730170,GMB,270
1033,Mozambique,Africa,1957,33.779,7038035,495.586833,MOZ,508
1541,Togo,Africa,1977,52.887,2308582,1532.776998,TGO,768
1339,Serbia,Europe,1987,71.218,9230783,15870.878510,SRB,688


## 📈 3️⃣ Basic Interactive Plotly Visualization

Before building the dashboard, let’s explore how we can visualize data interactively  
using **Plotly Express**.

In [27]:
# Create an interactive scatter plot
fig = px.scatter(
    df.query("year == 2007"),
    x="gdpPercap",
    y="lifeExp",
    size="pop",
    color="continent",
    hover_name="country",
    log_x=True,
    title="💡 Life Expectancy vs GDP per Capita (2007)",
)

fig.show()

## 🧱 4️⃣ Creating the Dash App Layout

Now we’ll create a **Dash** app layout consisting of:
- A **dropdown** to select the year  
- KPI metrics (average GDP, life expectancy, population)  
- Interactive charts that update dynamically

In [28]:
# Initialize Dash app
app = JupyterDash(__name__)

# Available years for dropdown
years = sorted(df["year"].unique())

# App layout
app.layout = html.Div([
    html.H1("🌍 Global Development Dashboard", style={"textAlign": "center"}),

    html.Div([
        html.Label("Select Year:", style={"fontWeight": "bold"}),
        dcc.Dropdown(
            id="year-dropdown",
            options=[{"label": y, "value": y} for y in years],
            value=2007,
            clearable=False
        )
    ], style={"width": "40%", "margin": "auto"}),

    html.Br(),

    html.Div(id="kpi-container", style={
        "display": "flex",
        "justifyContent": "space-around",
        "textAlign": "center",
        "marginBottom": "20px"
    }),

    dcc.Graph(id="scatter-plot"),
    dcc.Graph(id="continent-bar")
])

## ⚙️ 5️⃣ Adding Interactivity with Callbacks

We’ll use **Dash callbacks** to dynamically update:
- The KPI metrics  
- The scatter plot  
- The bar chart  
based on the selected year.

In [29]:
# Define callback for interactivity
@app.callback(
    [Output("kpi-container", "children"),
     Output("scatter-plot", "figure"),
     Output("continent-bar", "figure")],
    Input("year-dropdown", "value")
)
def update_dashboard(selected_year):
    filtered = df[df["year"] == selected_year]

    # Calculate KPIs
    avg_life = filtered["lifeExp"].mean()
    avg_gdp = filtered["gdpPercap"].mean()
    total_pop = filtered["pop"].sum()

    # KPI Cards
    kpis = [
        html.Div([
            html.H3("🌱 Life Expectancy"),
            html.H4(f"{avg_life:.2f} years")
        ], style={"backgroundColor": "#e8f5e9", "padding": "10px", "borderRadius": "10px", "width": "30%"}),

        html.Div([
            html.H3("💰 GDP per Capita"),
            html.H4(f"${avg_gdp:,.0f}")
        ], style={"backgroundColor": "#fff3e0", "padding": "10px", "borderRadius": "10px", "width": "30%"}),

        html.Div([
            html.H3("👥 Population"),
            html.H4(f"{total_pop/1e6:.1f} M")
        ], style={"backgroundColor": "#e3f2fd", "padding": "10px", "borderRadius": "10px", "width": "30%"})
    ]

    # Scatter plot
    scatter_fig = px.scatter(
        filtered,
        x="gdpPercap",
        y="lifeExp",
        size="pop",
        color="continent",
        hover_name="country",
        log_x=True,
        title=f"Life Expectancy vs GDP per Capita ({selected_year})"
    )

    # Bar chart by continent
    bar_fig = px.bar(
        filtered.groupby("continent")["pop"].sum().reset_index(),
        x="continent",
        y="pop",
        title="Total Population by Continent",
        color="continent"
    )

    return kpis, scatter_fig, bar_fig

## 🚀 6️⃣ Running the Dashboard

Now let’s launch the interactive dashboard directly inside this notebook using **JupyterDash**.  
You can interact with dropdowns and watch charts update in real time!

In [30]:
# 🚀 6️⃣ Running the Dashboard
"""
Final, fully working version for Dash >= 3.0
No deprecated methods, perfect for modern environments and portfolios.
"""

import warnings
warnings.filterwarnings('ignore')

from dash import Dash, dcc, html, Input, Output
import plotly.express as px

# Load sample dataset
df = px.data.gapminder()

# Initialize the Dash app
app = Dash(__name__)
server = app.server  # For deployment if needed

# Define layout
app.layout = html.Div([
    html.H1("🌍 Interactive Global Data Dashboard", 
            style={'textAlign': 'center', 'marginBottom': 20}),
    
    html.Label("Select a Year:", style={'fontWeight': 'bold'}),
    dcc.Slider(
        id='year-slider',
        min=df['year'].min(),
        max=df['year'].max(),
        value=2007,
        marks={str(year): str(year) for year in df['year'].unique()},
        step=None
    ),
    
    dcc.Graph(id='indicator-graphic', style={'marginTop': 20})
])

# Define callback
@app.callback(
    Output('indicator-graphic', 'figure'),
    Input('year-slider', 'value')
)
def update_graph(selected_year):
    filtered_df = df[df.year == selected_year]
    fig = px.scatter(
        filtered_df,
        x="gdpPercap", y="lifeExp",
        size="pop", color="continent", hover_name="country",
        log_x=True, size_max=60,
        title=f"🌐 Life Expectancy vs GDP per Capita ({selected_year})"
    )
    return fig

# ✅ Run the app (Dash 3.x style)
if __name__ == "__main__":
    print("🌐 Dashboard running at: http://127.0.0.1:8050/")
    app.run(debug=False, port=8050)

🌐 Dashboard running at: http://127.0.0.1:8050/


## 🏁 7️⃣ Final Summary & Closing

🎉 Congratulations — you've reached the end of the **Data Visualization with Python** journey!

In this notebook, you built a **fully interactive dashboard** using **Plotly Dash**, bringing together  
everything you've learned from previous notebooks — from static charts to fully dynamic web apps.

Throughout this series, you’ve learned how to:
- Create beautiful visualizations with **Matplotlib**, **Seaborn**, **Plotly**, and **Altair**
- Combine aesthetics with interactivity for better storytelling
- Build and customize dashboards with **Plotly Dash**
- Export and present insights in a professional, shareable format

🚀 This marks the completion of your **Data Visualization Portfolio Project** —  
you now have the tools to turn raw data into compelling visual stories.  
Whether you’re showcasing trends, building dashboards, or communicating insights —  
you can do it with confidence and clarity.  

**Thank you for following along, and congratulations on completing your data visualization journey!**  
✨ Keep exploring, keep creating, and keep visualizing.